In [1]:
import math
import pandas as pd
import kerouz_CNN as kr
import Embedding as yy
import numpy as np
from keras.preprocessing.text import Tokenizer # https://keras-cn.readthedocs.io/en/latest/preprocessing/text/
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
if __name__=='__main__':

    # adjustable parameter
    offset=10 # l1=l3=l2+offset. namely l1 refers to get length, l2 refers to average length, l3 refers to train length
    rm_symbols='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    embedding_dim=50


In [3]:
    # read dataset and dictionary
    data_train=pd.read_csv('../dataset/Train.csv')
    
    X_train=data_train['TEXT'].values
    Y_train=data_train['Label'].values
    

    #Y_train[Y_train == 5] = 1
    Y_train[Y_train == 6] = 1
    Y_train[Y_train == 11] = 1
    Y_train[Y_train == 12] = 1
    Y_train[Y_train == 13] = 1
 


    Y_train[Y_train == 7] = 2
    Y_train[Y_train == 15] = 2
    Y_train[Y_train == 8] = 2
    Y_train[Y_train == 14] = 2
    Y_train[Y_train == 18] = 2
    Y_train[Y_train == 9] = 2
    

    
    #Y_train[Y_train == 0] = 4
    Y_train[Y_train == 16] = 4
    Y_train[Y_train == 19] = 4

    
    Y_train[Y_train == 17] = 10
    
    
    
    '''
    以下有待商榷
    
    Y_train[Y_train == 2] = 1
    Y_train[Y_train == 4] = 1
    Y_train[Y_train == 10] = 1
    '''

    


    #Y_123 = Y_train[:]
    

    Y_train=to_categorical(Y_train,20)
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
    
    #data_test=pd.read_csv('../dataset/Test.csv')
   # X_test=data_test['TEXT'].values
    
    f='../dataset/glove.6B.50d.txt'
    
    emoji_map = pd.read_csv('../dataset/Mapping.csv')

In [4]:
    # remove special symbols and stopwords from train set
    X_rm=yy.corpus_pre(X_train)

    # segmentation
    tokenizer = Tokenizer(filters=rm_symbols, split=" ", lower=True) # filters：filter symbols that need to be removed lower：convert to lowercase
    tokenizer.fit_on_texts(X_rm) # Tokenizer read train set free of special symbols. Results are stored in tokenize handle.
    
    # vectorize. fill in and truncation
    l2 = math.ceil(sum([len(s.split(" ")) for s in X_rm])/len(X_rm)) # l2:average length
    l1 = l2+offset #get length
    X_pd,tokenizer = kr.toknz(X_rm, l1,tokenizer)

In [5]:
    #Dict that allocate an id(integer) to every word
    ind_dict=tokenizer.word_index

    #Dict that allocate an word vector to every word
    lookup_dict=yy.dict_pre(f)

    # generate weightMatrix according to dictionary
    W=yy.lookup(ind_dict,lookup_dict,embedding_dim)


In [6]:
    # train
    model=kr.model_training(len(ind_dict)+1, W, l2+offset, X_pd, Y_train, embed_dim=embedding_dim, epochs=3)
    print(model.predict_classes(X_pd[1:13])) #test on some sentences in the train data set

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            3643850   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 128)           32128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 4, 128)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 4, 128)            82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1, 128)            82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 128)           

## Predict on test set

In [14]:
    # Prediction on test set
    X_test_rm = yy.corpus_pre(X_test)
    X_test_pd,_ = kr.toknz(X_test_rm, l1,tokenizer)
    label_test = model.predict_classes(X_test_pd)
    #for i in range(500, 521, 1):
        #print(emoji_map['emoticons'][label_test[i]])
        #print(X_test[i])
    

In [15]:
label_test=to_categorical(label_test,20)

In [16]:
accuracy = 0
inaccuracy = []
for i in range(len(Y_test)):
    if (label_test[i,:] == Y_test[i,:]).all():
        accuracy += 1

accuracy = accuracy/len(Y_test)
        

In [17]:
accuracy

0.5473571428571429

In [11]:
    #loss, accuracy = model.evaluate(X_pd, Y_train, verbose=1)
    #print("Accuracy = %f  ;  loss = %f" % (accuracy, loss))

## Predict on user input

In [12]:
    '''
    user_str = input("input your sentence:")   
    #user_str = "I love you"
    X_user = np.array([str(user_str)])
    print(X_user[0])
    '''

'\nuser_str = input("input your sentence:")   \n#user_str = "I love you"\nX_user = np.array([str(user_str)])\nprint(X_user[0])\n'

In [13]:
    '''
    X_user_rm = yy.corpus_pre(X_user)
    X_user_pd,_ = kr.toknz(X_user_rm, l1,tokenizer)
    label_user = model.predict_classes(X_user_pd)
    print(emoji_map['emoticons'][label_user[0]])
    print(X_user[0]) 
    '''

"\nX_user_rm = yy.corpus_pre(X_user)\nX_user_pd,_ = kr.toknz(X_user_rm, l1,tokenizer)\nlabel_user = model.predict_classes(X_user_pd)\nprint(emoji_map['emoticons'][label_user[0]])\nprint(X_user[0]) \n"